In [1]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.prompts.prompts import SimpleInputPrompt
from transformers import BitsAndBytesConfig
import chromadb
import torch
from IPython.display import Markdown, display

c:\Users\GIGASYSTEM\miniconda3\envs\LlamaIdx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\GIGASYSTEM\miniconda3\envs\LlamaIdx\lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.get_or_create_collection('climate_report')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [3]:
documents = SimpleDirectoryReader("data").load_data()

c:\Users\GIGASYSTEM\miniconda3\envs\LlamaIdx\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [4]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )

In [6]:
torch.cuda.is_available()

True

In [7]:
llm = HuggingFaceLLM(
    context_window=2000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    model_kwargs={"quantization_config": quantization_config, "torch_dtype": torch.float16},
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2000}
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.30s/it]


In [8]:
embed_model =  HuggingFaceEmbedding(model_name="thenlper/gte-base")


In [9]:
service_context = ServiceContext.from_defaults(
    chunk_size=512,
    llm=llm,
    embed_model=embed_model,
    chunk_overlap=200
)

C:\Users\GIGASYSTEM\AppData\Local\Temp\ipykernel_1764\9962820.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [10]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

c:\Users\GIGASYSTEM\miniconda3\envs\LlamaIdx\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [13]:
query_engine = index.as_query_engine()
question = input("Q: ")
response = query_engine.query(question)
display(Markdown(f"{response}"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The excel file 'hate_examples' contains offensive samples from the SOCIAL BIASINFERENCE CORPUS -HATE (SBIC-H) dataset, where the offensive label is associated with a target group. These samples are set as hate class, while non-offensive samples are set as non-hate class. The samples that are labeled as offensive without a target group are not used.